# Ex13

In [1]:
! pip install --upgrade pip setuptools wheel
! pip install opencv-python
! pip install --upgrade implicit=='0.4.8'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import scipy
import implicit
import pandas as pd
from implicit.als import AlternatingLeastSquares

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.6
1.7.3
0.4.8


#### 데이터 준비와 전처리

In [4]:
rating_file_path='/content/drive/MyDrive/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
data = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(data)
data.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# 3점 이상만 남긴다.
data = data[data['ratings']>=3]
filtered_data_size = len(data)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# ratings 컬럼의 이름을 counts로 바꾼다.
data.rename(columns={'ratings':'counts'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
data['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어 온다.
movie_file_path='/content/drive/MyDrive/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
# 인기 많은 영화
movie_count = data.groupby('movie_id')['counts'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: counts, dtype: int64

영화 id 2858이 제일 인기가 많았다. 그렇다면 이 영화의 제목은 무엇일까?

In [10]:
movies.head(2790)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
2785,2854,Don't Look in the Basement! (1973),Horror
2786,2855,Nightmares (1983),Horror
2787,2856,I Saw What You Did (1965),Thriller
2788,2857,Yellow Submarine (1968),Animation|Musical


제일 인기가 많았던 영화는 'American Beauty'라는 것을 알 수 있다. 

영화를 추천 받기 위해서는

사용자 초기 정보 값을 입력해야 한다.

예를 들어, 넷플릭스 가입하는 동시 선호하는 영화를 몇 개 고르는 것과 같은 것이다.

'Back to the Future (1985)','Notting Hill (1999)', 'Toy Story (1995)', 'Titanic (1997)','Negotiator, The (1998)'를 좋아하는 영화로 선택


In [11]:
# Back_to_the_Futre, Toy_Story, Negotiation, Titanic, Notting_Hill

my_favorite = [1270, 2671, 3404, 2058, 1]
my_list = pd.DataFrame({'user_id': [9999]*5, 'movie_id': my_favorite, 'counts': [5]*5})
data = data.append(my_list)
data.tail(10)

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,9999,1270,5,NaN
1,9999,2671,5,NaN
2,9999,3404,5,NaN
3,9999,2058,5,NaN
4,9999,1,5,NaN


데이터의 쉽고 용이한 관리를 위해 indexing을 해주겠다.

In [12]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자이다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인해 보자. 
print(user_to_idx[9999]) 
print(movie_to_idx[1])

6039
40


In [13]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 보자. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거한다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해준다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해준다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,22,5,NaN
1,6039,1151,5,NaN
2,6039,1626,5,NaN
3,6039,791,5,NaN


### 모델링

In [14]:
# 매트릭스 만들기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [15]:
# implicit 라이브러리에서 권장하고 있는 부분이다.
import os
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=40, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/40 [00:00<?, ?it/s]

In [19]:
myukim, back_to_the_future = user_to_idx[9999], movie_to_idx[1270]
myukim_vector, back_to_the_future_vector = als_model.user_factors[myukim], als_model.item_factors[back_to_the_future]

In [20]:
myukim_vector

array([-4.56501126e-01,  3.06806058e-01,  4.54944856e-02,  2.38421202e-01,
        4.68992352e-01, -2.68016249e-01, -3.10929954e-01, -4.68259722e-01,
        4.06740606e-01,  4.85203266e-01,  8.32121670e-02, -3.03106099e-01,
       -4.97259945e-01, -6.37964547e-01,  3.04622680e-01, -5.39021969e-01,
       -1.81791767e-01,  6.53210640e-01,  2.00956538e-01,  2.70942561e-02,
        5.17858677e-02, -5.18791936e-02, -2.25053057e-01, -2.61379391e-01,
       -3.42237979e-01,  3.24937791e-01,  4.01372239e-02, -4.50136751e-01,
       -1.09297276e-01,  1.33190215e-01,  2.27736145e-01,  2.18706474e-01,
        2.64959395e-01,  1.40575618e-01, -1.85345877e-02, -1.87499687e-01,
       -2.63763517e-01,  3.41738254e-01,  2.32057929e-01, -6.75173521e-01,
       -5.09611607e-01, -3.82026076e-01,  1.78610846e-01,  1.05862662e-01,
        5.74165322e-02, -9.82424244e-02,  4.49782997e-01, -4.01732475e-01,
       -1.24432132e-01,  2.07048327e-01, -1.41121671e-01, -5.42628288e-01,
       -1.63792133e-01,  

In [21]:
back_to_the_future_vector

array([ 6.43208018e-03, -5.51897334e-03, -9.11962055e-03,  2.50011459e-02,
        2.07378659e-02, -1.09246960e-02,  3.53659107e-03, -1.58608425e-02,
        4.91886958e-02,  2.48959381e-03,  2.00588852e-02, -6.28616707e-03,
       -2.68826503e-02, -1.89378671e-02,  2.79357377e-02, -1.31917074e-02,
       -5.61291818e-05,  2.99522392e-02,  2.60638017e-02,  4.91030179e-02,
        1.20421182e-02,  3.73899750e-02,  2.76722331e-02, -1.87104568e-02,
        7.29099102e-03,  7.38871424e-03, -3.61429304e-02, -2.69225631e-02,
       -1.87227670e-02,  1.52160274e-02,  1.29979458e-02, -2.44875043e-03,
        4.20048945e-02, -3.17162042e-03, -2.76663415e-02, -1.17738172e-02,
       -3.65159102e-02,  1.92302149e-02,  1.74717102e-02,  2.75917281e-03,
       -1.27276359e-03, -3.38894478e-03,  4.39844467e-03,  2.03924309e-02,
        4.29340778e-03, -8.80708569e-04,  5.21259047e-02, -1.93118900e-02,
        2.50788052e-02,  1.61447357e-02, -7.35774729e-03, -1.37471156e-02,
       -7.84220826e-03,  

### 모델 평가하기

In [22]:
# myukim과 back_to_the_future를 내적하는 코드
np.dot(myukim_vector, back_to_the_future_vector)

0.8753883

선호도가 높게 나온다. 잘 훈련이 진행되었다는 것을 알 수 있다.

내가 선호했던 다른 영화도 이와 같은 결과를 가지는 지 궁금하다.

In [23]:
# movie_to_idx[1]은 toy story를 뜻한다.
toy_story = movie_to_idx[1]
toy_vector = als_model.item_factors[toy_story]
np.dot(myukim_vector, toy_vector)

0.7698914

토이스토리도 선호도가 높게 잘 나온다는 것을 알 수 있다.

### 비슷한 영화 and 추천 영화

#### 비슷한 영화

In [38]:
similar_movie = als_model.similar_items(back_to_the_future, N=15)
similar_movie

[(22, 1.0),
 (674, 0.34440017),
 (2827, 0.2563451),
 (13, 0.2522063),
 (3543, 0.2487783),
 (2824, 0.24733377),
 (615, 0.2466691),
 (1675, 0.24570818),
 (3370, 0.24502963),
 (3401, 0.24357004),
 (2765, 0.24247044),
 (2875, 0.2399736),
 (3565, 0.23879088),
 (3224, 0.23851028),
 (2714, 0.2381817)]

In [42]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
movie_id=[idx_to_movie[i[0]] for i in similar_movie]
print(movie_id)

[1270, 2011, 1743, 2918, 985, 1496, 2407, 718, 3294, 2743, 3171, 2281, 1316, 2536, 131]


1270은 Back_to_the_Future의 인덱스 번호다. 그래서 비슷한 정도가 1이 나온 것이다.

In [27]:
movies.head(1946)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
1941,2010,Metropolis (1926),Sci-Fi
1942,2011,Back to the Future Part II (1989),Comedy|Sci-Fi
1943,2012,Back to the Future Part III (1990),Comedy|Sci-Fi|Western
1944,2013,"Poseidon Adventure, The (1972)",Action|Adventure


비슷한 영화로 Back_to_the_Future2를 추천해주고 있다.

다음은 추천을 받아보자.

#### 시스템 추천 영화

In [30]:
user = user_to_idx[9999]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.28710002),
 (626, 0.22302878),
 (379, 0.21311197),
 (799, 0.20614022),
 (255, 0.20443882),
 (674, 0.18801814),
 (114, 0.17398337),
 (340, 0.17279443),
 (85, 0.17208341),
 (904, 0.17175482),
 (146, 0.16201994),
 (174, 0.15749191),
 (851, 0.1559447),
 (129, 0.15075892),
 (126, 0.15023936),
 (96, 0.1468999),
 (613, 0.14576909),
 (548, 0.14548454),
 (474, 0.1452814),
 (411, 0.14449377)]

similar 보다 더 유사도가 생각보다 낮게 나온다.

In [31]:
#원래 index로 바꿔주기
[idx_to_movie[i[0]] for i in movie_recommended]

[3114,
 2424,
 3256,
 2724,
 1923,
 2011,
 2353,
 1569,
 3255,
 1206,
 3418,
 2167,
 2761,
 1597,
 2396,
 1945,
 357,
 1282,
 3751,
 454]

In [32]:
# 첫번째로 추천해 준 영화
movies.head(3050)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy
3046,3115,Flawless (1999),Drama
3047,3116,Miss Julie (1999),Drama
3048,3117,Ride with the Devil (1999),Drama|Romance|War


Toy Story2를 추천해주고 있다. 그렇다면, 어떠한 영화가 이와 같이 추천을 해줬을까?

In [33]:
Toy_story2 = movie_to_idx[3114]
explain = als_model.explain(user, csr_data, itemid=Toy_story2)

In [34]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(1, 0.2584576048238112),
 (2671, 0.04674459129541232),
 (2058, 0.032664709614228335),
 (1270, 0.008281030502850104),
 (3404, -0.06168166799561124)]

전작품인 Toy Story1이 영향을 많이 주고 있다.

In [47]:
#두번째로 추천해준 영화
movies.head(2360)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
2355,2424,You've Got Mail (1998),Comedy|Romance
2356,2425,"General, The (1998)",Crime
2357,2426,"Theory of Flight, The (1998)",Comedy|Drama|Romance
2358,2427,"Thin Red Line, The (1998)",Action|Drama|War


In [49]:
YGM = movie_to_idx[2424]
explain = als_model.explain(user, csr_data, itemid=YGM)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(2671, 0.12596894602648695),
 (1270, 0.038445033868334824),
 (1, 0.030548215864230795),
 (2058, 0.023823080852747355),
 (3404, 0.002481863563968331)]

2671은 노팅힐의 인덱스 번호다. 노팅힐의 장르는 로맨틱 코미디이다. 이와 동일한 장르로서, 'You've Got Mail (1998)' 영화를 추천했다고 추측된다.

### 회고

날이 갈 수록 프로젝트는 어렵지만, 재미는 배가 되는 거 같다. 비록, 모든 것이 찍먹인 단계이지만 구현해낼 수 있다는 거에 의미를 두고 싶다. 고통은 당연 뒤따라 오고 있다. 하지만, 고통 뒤에 성장도 꼭 같이 오고 있다고 믿고 있다.


아쉬운 점

1. matrix를 좀 더 공부해야겠다.

